In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

In [2]:
dataset = load_digits()

In [3]:
X, y = dataset['data'], dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

### Задание
В скринкасте мы разобрали схему генерации признаков в стекинге, когда для тестовой выборки алгоритм заново переобучался на всей тренировочной выборке. Реализуйте схему, когда вместо этого производится агрегация ответов всех обученных на фолдах классификаторов на тестовой выборке при помощи усреднения.

Логика решения:
1. Создадим X_meta_test, заполним его нулями (по аналогии с X_meta_train);
2. Далее на каждом шаге, где мы обучаем folded_clf.fit (X_fold_train, y_fold_train) и его предсказания на X_fold_predict запихиваем в X_meta_train[predict_fold_index] добавим еще одну строку, где в X_meta_test будем добавлять предсказания вероятностей folded_clf на X_test. Их можно сразу складывать друг с другом или сохранить много массивов, тогда в конце их нужно будет все сложить, а потом делить на количество сплитов (количество массивов равно количеству сплитов в кросс - валидации);
3. После цикла останется только усреднить все эти массивы, это и будет наш X_meta_test.

За основу нужно взять следующий код:

def compute_meta_feature(clf, X_train, X_test, y_train, cv):

    """    Эта функция подсчитывает признаки для мета-классификатора.
    Они являются вероятностями классов при решении задачи многоклассовой классификации. 
    :arg clf: классификатор   
    :args X_train, y_train: обучающая выборка 
    :arg X_test: признаки тестовой выборки  
    :arg cv: класс, генерирующий фолды (KFold)
    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок    """
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = meta_clf.predict_proba(X_test)

    return X_meta_train, X_meta_test

In [ ]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора. 
    Они являются вероятностями классов при решении задачи многоклассовой классификации.

    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)

    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    X_meta_test = np.zeros((len(X_test), n_classes), dtype=np.float32)
    
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
        X_meta_test += folded_clf.predict_proba(X_test)
    
    X_meta_test = X_meta_test / cv.n_splits

    return X_meta_train, X_meta_test

## Задача

In [4]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [5]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test

In [7]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

ПОДСКАЗКА
- Во всех случаях, когда модель принимает random_state — обязательно указывайте его равным 42
- На разных версиях sklearn ответы могут отличаться
- В задании может понадобиться, а может не понадобиться нормализация и это нужно проверить во время решения задания.

### Задание
Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
- логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимое количество итераций — 2000
- логистическая регрессия с L2-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — multinomial, максимальное допустимое количество итераций — 2000
- случайный лес из 300 деревьев
- градиентный бустинг из 200 деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [9]:
lg_L1 = LogisticRegression(penalty='l1', C=0.001, solver='saga',
                          multi_class='ovr', max_iter=2000,
                          n_jobs=-1, random_state=42)
lg_L2 = LogisticRegression(penalty='l2', C=0.001, solver='saga',
                          multi_class='multinomial', max_iter=2000,
                          n_jobs=-1, random_state=42)
rfc = RandomForestClassifier(n_estimators=300, n_jobs=-1,
                           random_state=42)
gbc = GradientBoostingClassifier(n_estimators=200, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([lg_L1, lg_L2, rfc, gbc],
                                                                      X_train, X_test, y_train, cv)

100%|██████████| 4/4 [01:58<00:00, 29.51s/it]


In [10]:
meta_clf = LogisticRegression(penalty='none', solver='lbfgs', multi_class='auto', random_state=42)

In [11]:
compute_metric(meta_clf, X_train=stacked_features_train, y_train=y_train,
               X_test=stacked_features_test, y_test=y_test)

0.978096

### Задание
Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
- случайный лес из 300 деревьев
- случайный лес из 200 экстремальных деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [12]:
sf_train_2, sf_test_2 = generate_meta_features([RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42),
                                             ExtraTreesClassifier(n_estimators=200, n_jobs=-1, random_state=42)],
                                             X_train, X_test, y_train, cv)

100%|██████████| 2/2 [00:10<00:00,  5.09s/it]


In [13]:
compute_metric(meta_clf, X_train=sf_train_2, y_train=y_train,
               X_test=sf_test_2, y_test=y_test)

0.982421

### Задание
Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
- метод ближайшего соседа (k-NN) со стандартными параметрами
- случайный лес из 300 экстремальных деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [17]:
sf_train_3, sf_test_3 = generate_meta_features([KNeighborsClassifier(n_jobs = -1),
                                             ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42)],
                                             X_train, X_test, y_train, cv)


100%|██████████| 2/2 [00:06<00:00,  3.03s/it]


In [18]:
compute_metric(meta_clf, X_train=sf_train_3, y_train=y_train,
               X_test=sf_test_3, y_test=y_test)

C:\Users\obond\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.989904

### Задание
Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
- логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимоей количество итераций — 2000
- метод ближайшего соседа со стандартными параметрами
- случайный лес из 300 экстремальных деревьев
- AdaBoost со стандартными параметрами

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [19]:
sf_train_4, sf_test_4 = generate_meta_features([LogisticRegression(penalty='l1', C=0.001, solver='saga',
                                                                   multi_class='ovr', max_iter=2000,
                                                                   n_jobs=-1, random_state=42),
                                                KNeighborsClassifier(n_jobs = -1),
                                                ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42),
                                                AdaBoostClassifier(random_state=42)], 
                                               X_train, X_test, y_train, cv)


100%|██████████| 4/4 [00:19<00:00,  4.86s/it]


In [20]:
compute_metric(meta_clf, X_train=sf_train_4, y_train=y_train,
               X_test=sf_test_4, y_test=y_test)

0.987404

### Задание
Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
- случайный лес из 300 деревьев
- случайный лес из 300 экстремальных деревьев

Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).

Для корректной работы необходимо подправить код в функции compute_meta_feature. Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric. Количество фолдов = 10

In [29]:
cv_new = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [23]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train, y_train) # все изменения свелись к добавлению аргумента: y_train
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [30]:
sf_train_5, sf_test_5 = generate_meta_features([RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42),
                                             ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42)],
                                             X_train, X_test, y_train, cv_new)


100%|██████████| 2/2 [00:05<00:00,  2.88s/it]


In [28]:
compute_metric(meta_clf, X_train=sf_train_5, y_train=y_train,
               X_test=sf_test_5, y_test=y_test)

0.984228

In [35]:
meta_clf_new = GradientBoostingClassifier(random_state=42)

In [36]:
compute_metric(meta_clf_new, X_train=sf_train_5, y_train=y_train,
               X_test=sf_test_5, y_test=y_test)

0.984925

### Задание
Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
- случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
- случайный лес из 300 экстремальных деревьев

Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).

Для генерации фолдов используйте класс StratifiedKFold и поправленный Вами ранее код в функции compute_meta_feature.
Выполните разбиение на 3 фолда.

Как мета-алгортм используйте случайный лес из 100 экстремальных деревьев. Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [37]:
cv_new_2 = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [38]:
sf_train_6, sf_test_6 = generate_meta_features([RandomForestClassifier(n_estimators=300, n_jobs=-1,
                                                                       criterion = 'gini', max_depth = 24,
                                                                       random_state=42),
                                             ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42)],
                                             X_train, X_test, y_train, cv_new_2)


100%|██████████| 2/2 [00:03<00:00,  1.97s/it]


In [39]:
meta_clf_new_2 = ExtraTreesClassifier(n_estimators=100, n_jobs=-1, random_state=42)

In [40]:
compute_metric(meta_clf_new_2, X_train=sf_train_6, y_train=y_train,
               X_test=sf_test_6, y_test=y_test)

0.986498

### Задание
Обучите на тренировочной выборке следующие алгоритмы:
- случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
- случайный лес из 300 экстремальных деревьев
- логистическую регрессию со стандартными параметрами

Усредните их ответы на тестовой выборке методом сложения предсказаний и затем взятия функции argmax: answer = (prediction1 + prediction2 + prediction3).argmax(axis = 1).

Посчитайте качество, аналогично функции compute_metric (F1-score с макро-усреднением, округленный до 6 знака).

In [47]:
def compute_predicts(classifiers, X_train, y_train, X_test):
    predicts = []
    for clf in tqdm(classifiers):
        clf.fit(X_train, y_train)
        predicts.append(clf.predict_proba(X_test))
    return predicts

predicts = compute_predicts([RandomForestClassifier(n_estimators=300, criterion='gini',
                                                    max_depth=24, n_jobs=-1, random_state=42),
                             ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42), 
                             LogisticRegression()], X_train, y_train, X_test)

y_test_pred = np.sum(predicts, axis=0).argmax(axis=1)

print(np.round(f1_score(y_test, y_test_pred, average='macro'), 6))


 67%|██████▋   | 2/3 [00:01<00:00,  1.87it/s]C:\Users\obond\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

100%|██████████| 3/3 [00:01<00:00,  2.58it/s]

0.976259
